In [5]:
import pandas as pd
import numpy as np
from datetime import datetime, date
import geopandas as gpd
from geopandas import GeoDataFrame
import shapely
from shapely.geometry import Point
import matplotlib.pyplot as plt
import plotly.express as px
import folium
import json
import re
import openpyxl
import seaborn as sns
from matplotlib import cm
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import random
from pykml import parser
from os import path
import re
from scipy.spatial import ConvexHull, convex_hull_plot_2d

## estraggo i dati geografici 
wd = '/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/'
kml_file = '/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/input/doc.kml'
file_campate = '/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/input/TRcampate_TO_vento_diretti.xls'
file_mapping = '/Users/chiara/Desktop/git_repos/Terna/clustering/downscaling_R/input/ID_campate_TO_vento.xls'

with open(kml_file) as f:
  doc = parser.parse(f).getroot()
  
diz = {}

for e in doc.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
  coor = e.MultiGeometry.LineString.coordinates.text
  s = e.description.text
  a = re.search(r'STRNO', s)
  campata = re.sub('[STRNO</td>\n]', '', s[a.start():a.end()+30:1])
  diz[campata] = coor
  
lat_long_campate = pd.DataFrame.from_dict(diz,orient = 'index')
lat_long_campate.reset_index(inplace= True)
lat_long_campate.columns = ['campata', 'coordinates']
lat_long_campate['coordinates'] = lat_long_campate['coordinates'].apply(lambda x: x.split(','))
lat_long_campate2 = pd.DataFrame(lat_long_campate['coordinates'].to_list())
lat_long_campate2['campata'] = lat_long_campate['campata']
lat_long_campate2.columns = ['lon1', 'lat1', 'x', 'lat2', 'alt2', 'campata']
tmp = pd.DataFrame(lat_long_campate2['x'].apply(lambda x : x.split(' ')).to_list(), columns= ['alt1', 'lon2'])
tmp['campata'] = lat_long_campate2['campata']
final_lat_lon = tmp.merge(lat_long_campate2, on='campata')
final_lat_lon.drop(['x'], axis = 1, inplace = True)
final_lat_lon = final_lat_lon[['campata', 'lat1','lon1', 'alt1','lat2','lon2', 'alt2']]
final_lat_lon['linea'] = final_lat_lon['campata'].apply(lambda x : x.split('-')[0])


# GESTIONE ERRORI TYPO: ci sono più inconsistenze nei nomi delle campate/linee nel file KML rispetto ai nomi che si trovano nel file excel dei tempi di ritorno o rispetto al file excel che associa ad ogni campata il pixel 4x4km
# 
# 

errors = ["2341A1", "2342A1", "2420A1", "2421A1"]
mapping1 = {
"2341A1" : "23T41A1", 
"2342A1" : "23T42A1",
"2420A1" : "24T20A1",
"2421A1" : "24T21A1"}

final_lat_lon['campata'] = final_lat_lon['campata'].apply(lambda x : mapping1[x[:6]]+ x[6:] if x[:6] in errors else x)
final_lat_lon['linea'] = final_lat_lon['linea'].apply(lambda x : mapping1[x] if x in errors else x)

errors2 = ["24T21A1-115-D--","24T21A1-116-D--","24T21A1-117-D--","24T21A1-118-D--","24T21A1-119-D--","24T21A1-120-D--","24T21A1-121-D--","24T21A1-122-D--","24T21A1-123-D--","24T21A1-124-D--","24T21A1-125-D--","24T21A1-126-D--","24T21A1-126-B-D", "24T21A1-127-D--","24T21A1-128-D--"]
mapping2  = {"24T21A1-115-D--" : "24T21A1-115-DT--",
"24T21A1-116-D--" : "24T21A1-116-DT--",
"24T21A1-117-D--" : "24T21A1-117-DT--",
"24T21A1-118-D--" : "24T21A1-118-DT--",
"24T21A1-119-D--" : "24T21A1-119-DT--",
"24T21A1-120-D--" : "24T21A1-120-DT--",
"24T21A1-121-D--" : "24T21A1-121-DT--",
"24T21A1-122-D--" : "24T21A1-122-DT--",
"24T21A1-123-D--" : "24T21A1-123-DT--",
"24T21A1-124-D--" : "24T21A1-124-DT--",
"24T21A1-125-D--" : "24T21A1-125-DT--",
"24T21A1-126-D--" : "24T21A1-126-DT--",
"24T21A1-127-D--" : "24T21A1-127-DT--",
"24T21A1-128-D--" : "24T21A1-128-DT--",
"24T21A1-126-B-D" : "24T21A1-126-B-DT"
}
final_lat_lon['campata'] = final_lat_lon['campata'].apply(lambda x : mapping2[x] if x in errors2 else x)

errors3 = ["24T21A1-115---","24T21A1-116---","24T21A1-117---","24T21A1-118---","24T21A1-119---","24T21A1-120---","24T21A1-121---","24T21A1-122---","24T21A1-123---","24T21A1-124---","24T21A1-125---","24T21A1-126---","24T21A1-126---"]
mapping3  = {
"24T21A1-115---" : "24T21A1-115-TO--",
"24T21A1-116---" : "24T21A1-116-TO--",
"24T21A1-117---" : "24T21A1-117-TO--",
"24T21A1-118---" : "24T21A1-118-TO--",
"24T21A1-119---" : "24T21A1-119-TO--",
"24T21A1-120---" : "24T21A1-120-TO--",
"24T21A1-121---" : "24T21A1-121-TO--",
"24T21A1-122---" : "24T21A1-122-TO--",
"24T21A1-123---" : "24T21A1-123-TO--",
"24T21A1-124---" : "24T21A1-124-TO--",
"24T21A1-125---" : "24T21A1-125-TO--",
"24T21A1-126---" : "24T21A1-126-TO--",
"24T21A1-126---" : "24T21A1-126-TO--"
}
final_lat_lon['campata'] = final_lat_lon['campata'].apply(lambda x : mapping3[x] if x in errors3 else x)

mapping = pd.read_excel(file_mapping)
mapping.drop_duplicates('STRNO', inplace=True)
mapping = mapping.sort_values(['STRNO'],ignore_index=True)
final_lat_lon = final_lat_lon.sort_values(['campata'], ignore_index=True)
check = mapping[['STRNO']].merge(final_lat_lon['campata'], how = 'outer', left_on='STRNO', right_on='campata')
remaining_errors = pd.concat([check.loc[check.STRNO.isna()][['campata']].sort_values(by='campata', ignore_index=True),check.loc[check.campata.isna()][['STRNO']].sort_values(by='STRNO', ignore_index=True)],axis = 1)

errors4 = list(remaining_errors.campata)
mapping4 = dict(zip(remaining_errors.campata, remaining_errors.STRNO))
final_lat_lon['campata'] = final_lat_lon['campata'].apply(lambda x : mapping4[x] if x in errors4 else x)

check = mapping[['STRNO']].merge(final_lat_lon['campata'], left_on='STRNO', right_on='campata')
check.isna().sum()
## OK

official_campate = pd.read_excel(file_campate, header = None)
official_campate.rename(columns={0:'campata'}, inplace = True)
official_campate = official_campate[['campata']]
final_lat_lon = official_campate.merge(final_lat_lon, how = 'left', on = 'campata')

## CI SONO UFFICIALMENTE 28684 CAMPATE DA CONSIDERARE (SONO LE CAMPATE PER CUI TERNA CALCOLA I TEMPI DI RITORNO)
## nel file ID_campate_TO_vento.xls' c'erano invece 29439 campate totali. Noi teniamo come "ufficiali" le campate per cui si sono calcolati i TR

final_lat_lon.to_csv(wd + 'csv_files/spans_lat_lon.csv', sep='\t')

## SALVO IN UN CSV TUTTI I TYPO CHE HO MODIFICATO (dal kml file)
## total errors ha due colonne:
## - kml_file contiene il nome originale presente nel .kml (verosimilmente typo)
## - TR_excel contiene i nomi "ufficiali" presenti sia nel excel TRcampate_TO_vento_diretti.xls che in ID_campate_TO_vento.xls
## i nomi che ho usato nell'analisi sono quelli della colonna 'TR_excel'


total_errors = pd.concat([pd.DataFrame.from_dict(mapping1,orient = 'index'), pd.DataFrame.from_dict(mapping2,orient = 'index'), pd.DataFrame.from_dict(mapping3,orient = 'index')]).reset_index().rename(columns={'index':'kml_file', 0:'TR_excel'})
remaining_errors.rename(columns={'campata':'kml_file', 'STRNO':'TR_excel'}, inplace = True)
total_errors = pd.concat([total_errors, remaining_errors] )
total_errors.to_csv(wd +'extra/total_errors_typo.csv') ## to store all the errors
